# Retirement Planner

---

## Environment Setup

In [2]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
%matplotlib inline

In [3]:
# Set the random seed for resutls reproducibility (for testing purposes only)
np.random.seed(42)

---

## Portfolio Planner

In this activity, you will use the Alpaca API to grab historical data for a `60/40` portfolio using `SPY` to represent the stock portion and `AGG` to represent the bonds.

In [4]:
# Load .env enviroment variables
from dotenv import load_dotenv
load_dotenv('example.env')

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

# Data Collection

In this step, you will need to use the Alpaca api to fetch closing prices for the `SPY` and `AGG` tickers. Save the results as a pandas DataFrame

In [5]:
# Subset your tickers, then pull returns data:
# Set the ticker
ticker = ["SPY", "AGG"]

# Set timeframe to '1D'
timeframe = "1D"

# Set start and end datetimes of 1 year, between now and 365 days ago.
start_date = pd.Timestamp('2019-01-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2019-12-31', tz='America/New_York').isoformat()

# Get 1 year's worth of historical data for SPY and AGG
df = api.get_barset(
    ticker,
    timeframe,
    limit=None,
    start=start_date,
    end=end_date,
    after=None,
    until=None,
).df

# Drop unnecessary columns
df = df.drop(
    columns=['open', 'high', 'low', 'volume'],
    level=1
)

In [6]:
# Display sample data
df.head()

,AGG,SPY
,close,close
time,,
2019-01-02 00:00:00-05:00,106.585,249.89
2019-01-03 00:00:00-05:00,107.020,244.46
2019-01-04 00:00:00-05:00,106.695,252.41
2019-01-07 00:00:00-05:00,106.470,254.40
2019-01-08 00:00:00-05:00,106.420,256.65


---

## Monte Carlo Simulation

In this step, you will run Monte Carlo Simulations for your portfolio to model portfolio performance at different retirement ages. 

Complete the following steps:

1. Calculate the daily returns for the SPY and AGG closing prices.

2. Calculate volatility for both the SPY and AGG closing prices.

3. Find the last day's closing price for both stocks and save those as variables

4. Run a Monte Carlo Simulation of at least `100` iterations and generate at least `30` years of closing prices

**Hint:** There are `252` trading days per year, so the number of records to generate for each Monte Carlo run will be `252 days * 30 years`.

In [7]:
# Calculate the daily roi for the stocks
daily_returns = df.pct_change()


# Display sample data
daily_returns.head()

,AGG,SPY
,close,close
time,,
2019-01-02 00:00:00-05:00,NaN,NaN
2019-01-03 00:00:00-05:00,0.004081,-0.021730
2019-01-04 00:00:00-05:00,-0.003037,0.032521
2019-01-07 00:00:00-05:00,-0.002109,0.007884
2019-01-08 00:00:00-05:00,-0.000470,0.008844


In [8]:
avg_daily_return_agg = daily_returns.mean()
avg_daily_return_spy = daily_returns.mean()
avg_daily_return_spy

AGG  close    0.000213
SPY  close    0.001040
dtype: float64

In [9]:
# Compute daily volatility
std_dev_daily_return_agg = daily_returns.std()
std_dev_daily_return_spy = daily_returns.std() 
std_dev_daily_return_spy

AGG  close    0.00209
SPY  close    0.00782
dtype: float64

In [10]:
# Save the last day's closing price


spy_closing_price=df["SPY"]["close"][-1]
agg_closing_price=df["AGG"]["close"][-1]
spy_closing_price

321.92

In [11]:
# Setup the Monte Carlo Parameters
number_simulations = 10
number_records = 252 * 30
monte_carlo = pd.DataFrame()

In [12]:
# Run the Monte Carlo Simulation
# Set last closing prices of `TSLA` and `SPHD`
agg_last_price = df['AGG']['close'][-1]
spy_last_price = df['SPY']['close'][-1]

# Initialize empty DataFrame to hold simulated prices for each simulation
simulated_price_df = pd.DataFrame()
portfolio_cumulative_returns = pd.DataFrame()

# Run the simulation of projecting stock prices for the next trading year, `1000` times
for n in range(number_simulations):

    # Initialize the simulated prices list with the last closing price of `TSLA` and `SPHD`
    simulated_agg_prices = [agg_last_price]
    simulated_spy_prices = [spy_last_price]
    
    # Simulate the returns for 252 * 3 days
    for i in range(number_records):
        
        # Calculate the simulated price using the last price within the list
        simulated_agg_price = simulated_agg_prices[-1] * (1 + np.random.normal(avg_daily_return_agg, std_dev_daily_return_agg))
        simulated_spy_price = simulated_spy_prices[-1] * (1 + np.random.normal(avg_daily_return_spy, std_dev_daily_return_spy))
        
        # Append the simulated price to the list
        simulated_agg_prices.append(simulated_agg_price)
        simulated_spy_prices.append(simulated_spy_price)
    
    # Append the simulated prices of each simulation to DataFrame
    simulated_price_df["AGG prices"] = pd.Series(simulated_agg_prices)
    simulated_price_df["SPY prices"] = pd.Series(simulated_spy_prices)
    
    # Calculate the daily returns of simulated prices
    simulated_daily_returns = simulated_price_df.pct_change()
    
    # Set the portfolio weights (75% TSLA; 25% SPHD)
    weights = [0.6, 0.4]

    # Use the `dot` function with the weights to multiply weights with each column's simulated daily returns
    portfolio_daily_returns = simulated_daily_returns.dot(weights)
    
    # Calculate the normalized, cumulative return series
    portfolio_cumulative_returns[n] = (1 + portfolio_daily_returns.fillna(0)).cumprod()

# Print records from the DataFrame
portfolio_cumulative_returns

,0,1,2,3,4,5,6,7,8,9
0,1,1,1,1,1,1,1,1,1,1
1,"[1.0013770226622953, 1.0051554544669659]","[0.9979803945220365, 1.0033462206092387]","[1.001624229882177, 1.0092472455203059]","[1.001812259638988, 0.994012260955001]","[1.0003510358582945, 1.0002060694863872]","[0.9998971909145679, 1.0042588378406645]","[0.9994512124895607, 1.0147740609687872]","[1.0014770091700171, 1.0095264074194943]","[1.0019573511366717, 1.008945879090244]","[0.9982949666007395, 0.9997678305684397]"
2,"[1.0026180097126325, 1.0075096282446014]","[0.9981637177632406, 1.0065501847393346]","[1.001647106474304, 1.0113605706768614]","[1.0023120198084208, 1.0010235975431807]","[1.0016952961314738, 1.0005268133360485]","[1.0030124846623278, 1.0169207113443728]","[1.0014202860821872, 1.0138787879062068]","[0.9988697224666534, 1.0061029530752585]","[1.0005147796005822, 1.01407959637831]","[0.9969075279323243, 1.0046073401049898]"
3,"[1.0018527268378983, 1.0096546379661095]","[0.9970128143323242, 1.0013318028871527]","[1.001885776278393, 1.0071355956430041]","[1.000811290315543, 1.0030451610682631]","[1.0005646423359624, 1.0097288560080204]","[1.0049312223544324, 1.0124238364339093]","[1.0014008647441208, 1.0118928027169076]","[0.9974887126185884, 1.0058664408705746]","[1.0004445254356245, 1.0136543254032593]","[0.9965260542011604, 1.012498749861395]"
4,"[1.000925268061254, 0.9998651330527629]","[0.9948490161295273, 1.011109329655889]","[0.9995001321675174, 1.0084674291672064]","[0.9998019281274947, 1.001121457019815]","[1.0009651225057161, 1.0042825859127948]","[1.0050452539853316, 1.01119489556215]","[1.0001895555719227, 1.013933630839149]","[0.9952667731479963, 1.0079351946734427]","[1.0006484368010724, 1.0099042418519892]","[0.9977691045943122, 1.0054048676461296]"
5,"[0.9991073302316366, 0.9979622571940314]","[0.993396479296496, 1.0093044846234338]","[0.9993141732285071, 0.9963166771563292]","[0.9999310284243927, 1.0018031960500644]","[0.9989211591024943, 1.011329185700472]","[1.0062750713688806, 1.0119012609100404]","[0.9990071707726295, 1.0105562762050329]","[0.9918518014536034, 1.0078242818364975]","[1.0031877480173836, 1.0040640617733723]","[0.998015281302337, 0.997407783329983]"
6,"[1.0012124413118664, 0.9934955105243786]","[0.9925680141041144, 1.011737420910479]","[0.9962051915083379, 1.004129935210726]","[0.9993880303539989, 1.0102743542000903]","[1.0002230673092891, 1.0166701250906525]","[1.006148472338727, 1.0022507636381295]","[0.9993995352732268, 1.004642394055013]","[0.9908949301002324, 1.0146379295697925]","[1.0044963256266033, 1.0034482961806244]","[0.9989706634126567, 1.0046225387674832]"
7,"[0.9997787276468122, 0.9962134845870383]","[0.9943340636179359, 1.0049251574393587]","[0.9953510481216189, 1.015141923146907]","[0.9982927013691484, 1.0075005723801735]","[1.0018909684446007, 1.0213226194653349]","[1.0066383505847016, 1.0087856643475213]","[0.999425961224654, 1.002459855739515]","[0.9895034007944663, 1.0067892765522712]","[1.0060097748625516, 1.0036323729636913]","[0.9983327140263794, 1.0076831062276754]"
8,"[0.9987356076395165, 1.0016582439415112]","[0.9939003986225562, 1.0092120191264689]","[0.9966672029637553, 1.0161238991730244]","[1.002226003588142, 1.0031865371102875]","[1.0024918754832035, 1.0220380051265514]","[1.0076469758957316, 1.0036932747023244]","[1.000251913664671, 1.0055481184026287]","[0.9908253643710927, 1.0118396791113333]","[1.0031058101120087, 1.0003330629262202]","[0.9954752450031282, 1.0088135603244173]"
9,"[0.9996179291958789, 0.9939038930946775]","[0.9941498606304051, 1.0034453845083593]","[0.9970675374364376, 1.0106903353336365]","[1.0026345215081014, 1.0073996612090736]","[1.0035589570833183, 1.022204558949527]","[1.0104453548044634, 1.0126920562777602]","[1.0001750759847716, 1.0049941074685782]","[0.9927754116456294, 1.014751715007608]","[1.003960872629098, 0.9990070065732175]","[0.9954778879617625, 1.0072556976402194]"


In [25]:
simulated_agg_price = simulated_agg_prices[-1] * (1 + np.random.normal(avg_daily_return_agg, std_dev_daily_return_agg))
1 + np.random.normal(avg_daily_return_agg, std_dev_daily_return_agg)

array([1.00006851, 1.00458459])

In [26]:
avg_daily_return_agg

AGG  close    0.000213
SPY  close    0.001040
dtype: float64

In [ ]:
# Check that the simulation ran successfully
#portfolio_cumulative_returns.head()
pd.DataFrame({'1':[3,4,5], '2':[8,9,1]})

In [ ]:
pd.DataFrame({'1':[3,4,5], '2':[8,9,1]}).dot([1,2])

In [ ]:
# Visualize the Simulation
portfolio_cumulative_returns.plot()

In [ ]:
# Select the last row for the cumulative returns (cumulative returns at 30 years)
ending_cumulative_returns = portfolio_cumulative_returns.iloc[-1, :]
ending_cumulative_returns.head()

In [ ]:
# Display the 90% confidence interval for the ending returns
confidence_interval = ending_cumulative_returns.quantile(q=[0.025, 0.975])
confidence_interval

In [ ]:
# Visualize the distribution of the ending returns
plt.figure();
ending_cumulative_returns.plot(kind='hist', density=True, bins=10)
plt.axvline(confidence_interval.iloc[0], color='r')
plt.axvline(confidence_interval.iloc[1], color='r')

---

## Retirement Analysis

In this section, you will use the monte carlo model to answer the following retirement planning questions:

1. What are the expected cumulative returns at `30` years for the `10th`, `50th`, and `90th` percentiles?

2. Given an initial investment of `$20,000`, what is the expected return in dollars at the `10th`, `50th`, and `90th` percentiles?

3. Given the current projected annual income from the Plaid analysis, will a `4%` withdrawal rate meet or exceed that value at the `10th` percentile? Note: This is basically determining if retirement income is equivalent to current income.

4. How would a `50%` increase in the initial investment amount affect the `4%` retirement withdrawal? In other words, what happens if the initial investment had been bigger?

### What are the expected cumulative returns at 30 years for the 10th, 50th, and 90th percentiles?

In [ ]:
# Compute cumulative returns


### Given an initial investment of `$20,000`, what is the expected portfolio return in dollars at the 10th, 50th, and 90th percentiles?

In [ ]:
# Set initial investment

# Compute expected portfolio return


### Given the current projected annual income from the Plaid analysis, will a 4% withdraw rate from the retirement portfolio meet or exceed that value at the 10th percentile?

Note: This is effectively saying that 90% of the expected returns will be greater than the return at the 10th percentile, so this can help measure the uncertainty about having enough funds at retirement

In [ ]:
# Set Plaid's projected income

# Calculate the 4% withdrawal for the ending return as retirement income

# Determine if the retirement income meets or exceeds the current projected income


### How would a 50% increase in the initial investment amount affect the 4% retirement withdrawal?

In [ ]:
# Re-calculate the retirement income with a 50% increase in the initial investment amount


### Optional Challenge

Use the Monte Carlo data and calculate the cumulative returns at the `5%`, `50%`, and `95%` quartiles and plot this data as a line chart to see how the cumulative returns change over the life of the investment.

In this section, you need to calculate and plot the cumulative returns for the median and `90%` confidence intervals. This plot shows the expected cumulative returns for any given day between the first day and the last day of investment. 

In [ ]:
# Compute projected returns

# Display sample data


In [ ]:
# Plot the cumulative returns over time


In [ ]:
# Compute portfolio performance over time

# Plot projected performance over time
